In [1]:
# read it in to inspect it
with open('poem.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  2368921


In [3]:
print(text[:1000])

বজাও রে মোহন বাঁশি।
সারা দিবসক
বিরহদহনদুখ,
মরমক তিয়াষ নাশি।
রিঝমনভেদন
বাঁশরিবাদন
কঁহা শিখলি রে কান?
হানে থিরথির
মরমঅবশকর
লহু লহু মধুময় বাণ।
ধসধস করতহ
উরহ বিয়াকুলু,
ঢুলু ঢুলু অবশনয়ান ;
কত কত বরষক
বাত সোঁয়ারয়,
অধীর করয় পরান।
কত শত আশা
পূরল না বঁধু,
কত সুখ করল পয়ান।
পহু গো কত শত
পীরিতযাতন
হিয়ে বিঁধাওল বাণ।
হৃদয় উদাসয়,
নয়ন উছাসয়
দারুণ
মধুময় গান।
সাধ যায় বঁধূ,
যমুনাবারিম
ডারিব
দগধপরান।
সাধ যায় পহু,
রাখি চরণ তব
হৃদয়মাঝ
হৃদয়েশ,
হৃদয়জুড়াওন
বদনচন্দ্র তব
হেরব
জীবনশেষ।
সাধ যায়, ইহ
চন্দ্রমকিরণে
কুসুমিত
কুঞ্জবিতানে
বসন্তবায়ে
প্রাণ মিশায়ব
বাঁশিক সুমধুর গানে।
প্রাণ ভৈবে মঝু
বেণুগীতময়,
রাধাময় তব
বেণু।
জয় জয় মাধব,
জয় জয় রাধা,
চরণে
প্রণমে ভানু।
শুনহ শুনহ বালিকা,
রাখ কুসুমমালিকা,
কুঞ্জ কুঞ্জ ফেরনু সখি শ্যামচন্দ্র নাহি রে।
দুলই কুসুমমুঞ্জরী,
ভমর ফিরই গুঞ্জরি,
অলস যমুনা বহয়ি যায় ললিত গীত গাহি রে।
শশিসনাথ যামিনী,
বিরহবিধুর কামিনী,
কুসুমহার ভইল ভার— হৃদয় তার দাহিছে।
অধর উঠই কাঁপিয়া
সখিকরে কর আপিয়া,
কুঞ্জভবনে পাপিয়া কাহে গীত গাহিছে।
মৃদু সমীর সঞ্চলে
হরয়ি শিথিল অঞ্চলে,
চকিত হৃদয় চঞ্চলে কাননপথ চাহি রে।
কুঞ্জপান

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"'(),-.12679:;?BCFHJLMNORTW[]_abcdefghiklmnoprstuvwxy|ű̶।॥ঁংঃঅআইঈউঊঋএঐওঔকখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহািীুূৃেৈোৌ্ৎড়ঢ়য়০১২৩৪৫৬৭৮৯‍–—‘’“”
139


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("আমার জুতা"))
print(decode(encode("আমার জুতা")))

[65, 99, 107, 101, 1, 82, 110, 90, 107]
আমার জুতা


In [1]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

c:\users\reinh\appdata\local\programs\python\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'encode' is not defined

In [3]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

NameError: name 'encode' is not defined

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('poem.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


c:\users\reinh\appdata\local\programs\python\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.219275 M parameters
step 0: train loss 5.0452, val loss 5.0452
step 100: train loss 2.8748, val loss 2.9285
step 200: train loss 2.7750, val loss 2.8280
step 300: train loss 2.7251, val loss 2.7709
step 400: train loss 2.6808, val loss 2.7317
step 500: train loss 2.6326, val loss 2.6847
step 600: train loss 2.5852, val loss 2.6520
step 700: train loss 2.5669, val loss 2.6216
step 800: train loss 2.5186, val loss 2.5683
step 900: train loss 2.5049, val loss 2.5502
step 1000: train loss 2.4774, val loss 2.5189
step 1100: train loss 2.4364, val loss 2.4881
step 1200: train loss 2.4233, val loss 2.4840
step 1300: train loss 2.3943, val loss 2.4442
step 1400: train loss 2.3670, val loss 2.4275
step 1500: train loss 2.3714, val loss 2.4106
step 1600: train loss 2.3517, val loss 2.4093
step 1700: train loss 2.3273, val loss 2.3812
step 1800: train loss 2.3228, val loss 2.3597
step 1900: train loss 2.3104, val loss 2.3548
step 2000: train loss 2.2901, val loss 2.3532
step 2100: train loss 2.